In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("<style>.output_result { max-width:90% !important; }</style>"))

In [2]:
import json 
from pprint import pprint 
import pandas as pd
import numpy as np
from pathlib import Path
import os
import re

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [4]:
def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.
  Args:
    dir_path (str or pathlib.Path): target directory
  
  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} JSON files in '{dirpath}'.")

In [5]:
data_path = Path("ALGORITM/")
walk_through_dir(data_path)

There are 0 directories and 628 JSON files in 'ALGORITM'.


In [6]:
# Get all file paths - все файлы в папке ALGORITM
file_path_list = list(data_path.glob("*.json"))

i = 0
file_id=list()
print(type(file_id))
for file in file_path_list: 
   
    print(f"{i+1} file path: {file_path_list[i]}")
    fp = str(file_path_list[i])
    file_id.append(fp[-10:-5])
    i+=1

<class 'list'>
1 file path: ALGORITM\data_for_algoritm_11834.json
2 file path: ALGORITM\data_for_algoritm_12729.json
3 file path: ALGORITM\data_for_algoritm_13579.json
4 file path: ALGORITM\data_for_algoritm_13580.json
5 file path: ALGORITM\data_for_algoritm_14208.json
6 file path: ALGORITM\data_for_algoritm_14457.json
7 file path: ALGORITM\data_for_algoritm_14770.json
8 file path: ALGORITM\data_for_algoritm_14771.json
9 file path: ALGORITM\data_for_algoritm_14772.json
10 file path: ALGORITM\data_for_algoritm_14775.json
11 file path: ALGORITM\data_for_algoritm_14781.json
12 file path: ALGORITM\data_for_algoritm_14784.json
13 file path: ALGORITM\data_for_algoritm_14786.json
14 file path: ALGORITM\data_for_algoritm_14787.json
15 file path: ALGORITM\data_for_algoritm_14793.json
16 file path: ALGORITM\data_for_algoritm_14798.json
17 file path: ALGORITM\data_for_algoritm_14803.json
18 file path: ALGORITM\data_for_algoritm_14806.json
19 file path: ALGORITM\data_for_algoritm_14808.json
20 fil

In [9]:
k = 0
load_data = pd.DataFrame()
ld = pd.DataFrame()
for file in file_path_list: 
    file_path = 'ALGORITM/data_for_algoritm_' + file_id[k] + '.json'
    load_file = 'da_'+ file_id[k]
    with open(file_path, encoding='utf-8') as f:
        load_file = json.load(f) 
    df2 = pd.json_normalize(load_file, max_level=3).melt()
    
 # вывод таблиц с размерами и свойствами коробок, а их дофига...
    df4 = pd.json_normalize(df2.loc[df2["variable"] == "data_result.boxes","value"].iloc[0], max_level=0).melt()
    cargo = pd.DataFrame()
    cargo['size'] = df4.loc[df4['variable'] == 'size', ['value']].reset_index(drop=True)
    stacking = df4.loc[df4['variable'] == 'stacking', ['value']].reset_index(drop=True)
    turnover = df4.loc[df4['variable'] == 'turnover', ['value']].reset_index(drop=True)
    cargo['turnover'] = turnover['value']
    cargo['stacking'] = stacking['value'] 
    #display(cargo)  # для вывода всех таблиц с размерами и свойствами коробок разкомментируйте эту строку

 # создание таблицы свойств карго со всех файлов  
    load_data = [{'load_id':file_id[k],'density_percent':df2.loc[df2["variable"] == "data_result.cargo_space.calculation_info.density_percent","value"].iloc[0], 
                 'filling_space_percent':df2.loc[df2["variable"] == "data_result.cargo_space.calculation_info.filling_space_percent","value"].iloc[0], 
                 'loading_width':df2.loc[df2["variable"] == "data_result.cargo_space.loading_size.width","value"].iloc[0],
                 'loading_height':df2.loc[df2["variable"] == "data_result.cargo_space.loading_size.height","value"].iloc[0], 
                 'loading_length':df2.loc[df2["variable"] == "data_result.cargo_space.loading_size.length","value"].iloc[0], 'boxes_qty':len(cargo)}]
              
    ld = pd.concat((ld, pd.DataFrame(load_data)))
    k+=1

In [10]:
 # вывод таблицы свойств карго со всех файлов 
ld.reset_index(drop=True)

,load_id,density_percent,filling_space_percent,loading_width,loading_height,loading_length,boxes_qty
0,11834,81.76416,81.72327,800.0,2000.0,1200.0,270
1,12729,82.50000,51.56250,800.0,2000.0,1200.0,33
2,13579,94.44444,64.24381,2500.0,2100.0,7500.0,17
3,13580,74.02778,67.29798,800.0,1650.0,1200.0,33
4,14208,100.00000,58.26317,2490.0,2590.0,13490.0,33
5,14457,92.20825,90.27382,2480.0,2860.0,13600.0,966
6,14770,55.72132,44.63278,2400.0,2500.0,6100.0,7
7,14771,82.70950,72.29493,2490.0,2590.0,13490.0,54
8,14772,58.69138,47.83901,2450.0,2650.0,8800.0,18
9,14775,55.72132,44.63278,2400.0,2500.0,6100.0,7
